# Contents
[Clean SQL database](#cleaning-data-from-the-sql-database-from-aws)  
[Clean S3 bucket PDF data](#data-cleaning-of-card-details-from-the-s3-bucket)  
[Code to GET API data](#checking-code-to-get-api-data)  
[Clean API data](#reviewing-and-cleaning-the-api-data)  
[Clean S3 bucket CSV data](#extracting-s3-object-using-boto3)


### Cleaning data from the SQL database from AWS

In [63]:
from data_extraction import DataExtractor
from data_cleaning import DataCleaning
from database_utils import DatabaseConnector
import pandas as pd
import numpy as np

db = DatabaseConnector()
table = DataExtractor()


In [64]:
df = table.read_rds_table(db, 'legacy_users')
df = df.drop('index',axis=1)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     15320 non-null  object
 1   last_name      15320 non-null  object
 2   date_of_birth  15320 non-null  object
 3   company        15320 non-null  object
 4   email_address  15320 non-null  object
 5   address        15320 non-null  object
 6   country        15320 non-null  object
 7   country_code   15320 non-null  object
 8   phone_number   15320 non-null  object
 9   join_date      15320 non-null  object
 10  user_uuid      15320 non-null  object
dtypes: object(11)
memory usage: 1.3+ MB


In [66]:
df.head()

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [67]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     15320 non-null  object
 1   last_name      15320 non-null  object
 2   date_of_birth  15320 non-null  object
 3   company        15320 non-null  object
 4   email_address  15320 non-null  object
 5   address        15320 non-null  object
 6   country        15320 non-null  object
 7   country_code   15320 non-null  object
 8   phone_number   15320 non-null  object
 9   join_date      15320 non-null  object
 10  user_uuid      15320 non-null  object
dtypes: object(11)
memory usage: 1.3+ MB
None


In [68]:
# First process the the date columns 
# Converting any date columns into the date format
import pandas as pd
from dateutil.parser import parse
for col in df.columns:  
    if 'date' in col:
        def try_parsing_date(text):
            try:
                return parse(text)
            except Exception:
                return pd.NaT
        
        df[col] = df[col].apply(try_parsing_date)
        df[col] = pd.to_datetime(df[col], errors='coerce')



In [69]:
from datetime import datetime
# Calculate the current date
current_date = datetime.now()
# Subset the data to show individuals older than 70 years
subset_older_than_70 = df[current_date.year - df['date_of_birth'].dt.year > 80]
print(subset_older_than_70)

      first_name last_name date_of_birth                      company  \
1            Guy     Allen    1940-12-01                      Fox Ltd   
21          Jörg  Hoffmann    1940-11-04                        Höfig   
66        Olivia  Williams    1940-10-23  Sanderson, Singh and Wilson   
77      Klaus-D.      Gute    1942-06-21        Conradi GmbH & Co. KG   
84       Dominic    Potter    1940-01-19                 Powell-Carey   
...          ...       ...           ...                          ...   
15274       Joan      Reed    1942-10-24                    Frost PLC   
15289    Stephen     Davis    1940-08-30    Graham, Fisher and Gibson   
15292    Stephen      Ward    1940-07-22                  Smith-Brown   
15308    Stephen    Hudson    1940-09-13                   Harris PLC   
15317    Stephen  Losekann    1940-10-09                      Rosenow   

                        email_address  \
1        rhodesclifford@henderson.com   
21             zgehringer@beckmann.de   


In [70]:
#Removing NA rows of data
#Further inspection show all NA rows in date_of_birth include non-null rows but are randomised.
nat_indexes = df.index[df['date_of_birth'].isna()].tolist()
print(nat_indexes)
na_list = df.iloc[nat_indexes]
print(na_list)

[752, 866, 1022, 1046, 1805, 2103, 2437, 2739, 2764, 2995, 3536, 4984, 5306, 5307, 6420, 6920, 7737, 8386, 9013, 10013, 10211, 10224, 10360, 10988, 11366, 11443, 11598, 11761, 11864, 12092, 12177, 12584, 13111, 13855, 14101, 14499]
       first_name   last_name date_of_birth     company email_address  \
752    PYCLKLLC7I  W350SCUD6R           NaT  R7IZUNSQX0    3Q791B3VIY   
866          NULL        NULL           NaT        NULL          NULL   
1022         NULL        NULL           NaT        NULL          NULL   
1046   GI4C78KWH0  UTB5PPYFG8           NaT  CA1XGS8GZW    7HSZB429UK   
1805         NULL        NULL           NaT        NULL          NULL   
2103         NULL        NULL           NaT        NULL          NULL   
2437         NULL        NULL           NaT        NULL          NULL   
2739         NULL        NULL           NaT        NULL          NULL   
2764         NULL        NULL           NaT        NULL          NULL   
2995   DPAJNJL6PR  B8ZGN8ZJ84         

In [71]:
# Double checking if the parse and to_datetime function worked on dates that had string e.g. October 
# Anti join show na_list removes the randomised and unparsable dates and leaves the good ones in
result = df[df['date_of_birth'].astype(str).str.contains('[A-Za-z]', regex=True, na=False)]
anti_join_df = result.merge(na_list, left_index=True, right_index=True, how='left', indicator=True)
anti_join_result = anti_join_df[anti_join_df['_merge'] == 'left_only']
print(result)
print(anti_join_result)

       first_name   last_name date_of_birth     company email_address  \
752    PYCLKLLC7I  W350SCUD6R           NaT  R7IZUNSQX0    3Q791B3VIY   
866          NULL        NULL           NaT        NULL          NULL   
1022         NULL        NULL           NaT        NULL          NULL   
1046   GI4C78KWH0  UTB5PPYFG8           NaT  CA1XGS8GZW    7HSZB429UK   
1805         NULL        NULL           NaT        NULL          NULL   
2103         NULL        NULL           NaT        NULL          NULL   
2437         NULL        NULL           NaT        NULL          NULL   
2739         NULL        NULL           NaT        NULL          NULL   
2764         NULL        NULL           NaT        NULL          NULL   
2995   DPAJNJL6PR  B8ZGN8ZJ84           NaT  0YJ2FRMDB4    O5Q6D7FDAF   
3536   E5139W1FJ3  X88YLZX0JX           NaT  648S19YUFF    BYU3R2LQF8   
4984         NULL        NULL           NaT        NULL          NULL   
5306   XKDVQD7BH2  4YSEX8AY1Z           NaT  GKJZ58

In [72]:
# So to conclude, all nulls in date of birth removed, this also removes randomised records of data
df = df.drop(nat_indexes)

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     15284 non-null  object        
 1   last_name      15284 non-null  object        
 2   date_of_birth  15284 non-null  datetime64[ns]
 3   company        15284 non-null  object        
 4   email_address  15284 non-null  object        
 5   address        15284 non-null  object        
 6   country        15284 non-null  object        
 7   country_code   15284 non-null  object        
 8   phone_number   15284 non-null  object        
 9   join_date      15284 non-null  datetime64[ns]
 10  user_uuid      15284 non-null  object        
dtypes: datetime64[ns](2), object(9)
memory usage: 1.4+ MB


In [74]:
# This shows no NA values are left. Following scripts would be to look for and handle incorrect data
na_counts = df.isna().sum()
print(na_counts)

first_name       0
last_name        0
date_of_birth    0
company          0
email_address    0
address          0
country          0
country_code     0
phone_number     0
join_date        0
user_uuid        0
dtype: int64


In [75]:
# Next fix the phone numbers
# The below processing of US, UK and Germany has been combined into this single cell
us_indexes = df[df['country'] == "United States"].index
other_indexes = df[df['country'] != "United States"].index

df.loc[us_indexes, 'phone_number'] = df.loc[us_indexes, 'phone_number'].str.replace(r'(\+\d{1}|x\d+|001-|[\.\-\(\)])', '', regex=True)
df.loc[us_indexes, 'phone_number'] = df.loc[us_indexes, 'phone_number'].apply(lambda x: np.nan if len(str(x)) != 11 else x)

df.loc[other_indexes, 'phone_number'] = df.loc[other_indexes, 'phone_number'].str.replace(r'(\+\d{2})|\s+|[(|)]', '', regex=True)
df.loc[other_indexes, 'phone_number'] = df.loc[other_indexes, 'phone_number'].apply(lambda x: np.nan if len(str(x)) != 11 else x)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     15284 non-null  object        
 1   last_name      15284 non-null  object        
 2   date_of_birth  15284 non-null  datetime64[ns]
 3   company        15284 non-null  object        
 4   email_address  15284 non-null  object        
 5   address        15284 non-null  object        
 6   country        15284 non-null  object        
 7   country_code   15284 non-null  object        
 8   phone_number   9089 non-null   object        
 9   join_date      15284 non-null  datetime64[ns]
 10  user_uuid      15284 non-null  object        
dtypes: datetime64[ns](2), object(9)
memory usage: 1.9+ MB


In [77]:
#Review country_code
df['country_code'].value_counts()

country_code
GB     9365
DE     4708
US     1205
GGB       6
Name: count, dtype: int64

In [78]:
df.loc[df['country_code'] == "GGB", 'country_code'] = "GB"

In [79]:
# Next country - all good
df['country'].value_counts()

country
United Kingdom    9371
Germany           4708
United States     1205
Name: count, dtype: int64

In [80]:
# assumption that the address column is correct, could separate postcode and cross-reference on google maps
print(df['address'][0])

Zimmerstr. 1/0
59015 Gießen


In [81]:
# Remove the new line and replace with comma
df['address'] = df['address'].str.replace('\n', ',')
print(df['address'])

0                             Zimmerstr. 1/0,59015 Gießen
1          Studio 22a,Lynne terrace,McCarthymouth,TF0 9GH
2                        92 Ann drive,Joanborough,SK0 6LR
3                    19 Robinson meadow,New Tracy,W22 2QG
4                     3 White pass,Hunterborough,NN96 4UE
                               ...                       
15315         Studio 41I,Jones lodge,Oliviaborough,E8 3DU
15316                 530 Young parkway,Millsfurt,L4G 7NX
15317    Viviane-Fritsch-Straße 3/5,15064 Bad Liebenwerda
15318     660 Ross Falls Suite 357,Anthonymouth, MA 09610
15319         Studio 4,Hancock road,Phillipsview,ST1X 3XB
Name: address, Length: 15284, dtype: object


In [82]:
#All email addresses follow a consistent pattern
df['email_address'].str.contains('@.*\.', regex=True).value_counts()

email_address
True    15284
Name: count, dtype: int64

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     15284 non-null  object        
 1   last_name      15284 non-null  object        
 2   date_of_birth  15284 non-null  datetime64[ns]
 3   company        15284 non-null  object        
 4   email_address  15284 non-null  object        
 5   address        15284 non-null  object        
 6   country        15284 non-null  object        
 7   country_code   15284 non-null  object        
 8   phone_number   9089 non-null   object        
 9   join_date      15284 non-null  datetime64[ns]
 10  user_uuid      15284 non-null  object        
dtypes: datetime64[ns](2), object(9)
memory usage: 1.9+ MB


In [84]:

print(df['first_name'].str.contains(r'\d', regex=True).value_counts())
print(df['last_name'].str.contains(r'\d', regex=True).value_counts())
print(df['company'].str.contains(r'\d', regex=True).value_counts())

first_name
False    15284
Name: count, dtype: int64
last_name
False    15284
Name: count, dtype: int64
company
False    15284
Name: count, dtype: int64


In [85]:
df['first_name'].value_counts()

first_name
Michael       57
Amanda        52
Peter         51
Jennifer      50
Andrew        50
              ..
Kristopher     1
Guido          1
Darlene        1
Adem           1
Athanasios     1
Name: count, Length: 2162, dtype: int64

In [86]:
#Below was the analysis of numbers from different countries

In [87]:
# Next fix the phone numbers
df['country'].value_counts()

country
United Kingdom    9371
Germany           4708
United States     1205
Name: count, dtype: int64

In [88]:
#

In [89]:
#10 digits
us = df[df['country'] == 'United States']
print(us)
print(us.info())

      first_name last_name date_of_birth                        company  \
5          David    Torres    1949-08-12                   Yang-Stewart   
9          Emily     Jones    1992-10-09                Serrano-Leblanc   
16        Sierra   Spencer    1988-11-02       Watson, Davis and Cooper   
27         Bryan   Nielsen    1979-07-29                    Drake-Clark   
35       Melissa    Garcia    1946-06-30  Nelson, Williams and Martinez   
...          ...       ...           ...                            ...   
15275     George      Cook    1974-01-10                     Miller LLC   
15289    Stephen     Davis    1940-08-30      Graham, Fisher and Gibson   
15301    Stephen  Gonzales    2006-04-17                    Marquez Ltd   
15311    Stephen  Alvarado    2006-07-27                  Wallace-Smith   
15318    Stephen    Rivera    1952-06-04          Taylor, Fry and Jones   

                       email_address  \
5              mwilliams@nichols.org   
9                 t

In [90]:
# For the USA, typically follow XXX-XXX-XXXX pattern
us.loc[:,'phone_number'] = us['phone_number'].str.replace(r'(\+\d{1}|x\d+|001-|[\.\-\(\)])','',regex=True)
us.loc[:,'valid_phone'] = us['phone_number'].str.len() == 10
print(us['valid_phone'].value_counts())
print(us)


valid_phone
False    1205
Name: count, dtype: int64
      first_name last_name date_of_birth                        company  \
5          David    Torres    1949-08-12                   Yang-Stewart   
9          Emily     Jones    1992-10-09                Serrano-Leblanc   
16        Sierra   Spencer    1988-11-02       Watson, Davis and Cooper   
27         Bryan   Nielsen    1979-07-29                    Drake-Clark   
35       Melissa    Garcia    1946-06-30  Nelson, Williams and Martinez   
...          ...       ...           ...                            ...   
15275     George      Cook    1974-01-10                     Miller LLC   
15289    Stephen     Davis    1940-08-30      Graham, Fisher and Gibson   
15301    Stephen  Gonzales    2006-04-17                    Marquez Ltd   
15311    Stephen  Alvarado    2006-07-27                  Wallace-Smith   
15318    Stephen    Rivera    1952-06-04          Taylor, Fry and Jones   

                       email_address  \
5      

C:\Users\Janoh\AppData\Local\Temp\ipykernel_2852\3246566554.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us.loc[:,'valid_phone'] = us['phone_number'].str.len() == 10


In [91]:
#Error check - ensure all numbers have exactly 10 digits
us.loc[:,'valid_phone'] = us['phone_number'].str.len() == 10
print(us['valid_phone'].value_counts())

valid_phone
False    1205
Name: count, dtype: int64


In [92]:
#Germany have 11 digits
germany = df[df['country'] == "Germany"]
print(germany.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4708 entries, 0 to 15317
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     4708 non-null   object        
 1   last_name      4708 non-null   object        
 2   date_of_birth  4708 non-null   datetime64[ns]
 3   company        4708 non-null   object        
 4   email_address  4708 non-null   object        
 5   address        4708 non-null   object        
 6   country        4708 non-null   object        
 7   country_code   4708 non-null   object        
 8   phone_number   2615 non-null   object        
 9   join_date      4708 non-null   datetime64[ns]
 10  user_uuid      4708 non-null   object        
dtypes: datetime64[ns](2), object(9)
memory usage: 441.4+ KB
None


In [93]:
# Some german numbers are 10 digits. Replace with NaN
germany.loc[:,'phone_number'] = germany['phone_number'].str.replace(r'(\+\d{2})|\s+|[(|)]','', regex= True)

In [94]:
#Error check - ensure all numbers have exactly 10 digits
germany.loc[:,'valid_phone'] = germany['phone_number'].str.len() == 11
germany.loc[~germany['phone_number'].apply(lambda x: len(str(x)) == 11), 'phone_number'] = np.nan
print(germany['valid_phone'].value_counts())


valid_phone
True     2615
False    2093
Name: count, dtype: int64


C:\Users\Janoh\AppData\Local\Temp\ipykernel_2852\787672041.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  germany.loc[:,'valid_phone'] = germany['phone_number'].str.len() == 11


In [95]:
#Lastly the UK
uk = df[df['country'] == "United Kingdom"]
print(uk)

      first_name last_name date_of_birth                       company  \
1            Guy     Allen    1940-12-01                       Fox Ltd   
2          Harry  Lawrence    1995-08-02     Johnson, Jones and Harris   
3         Darren   Hussain    1972-09-23                   Wheeler LLC   
4          Garry     Stone    1952-12-20                    Warner Inc   
6           Anne    Morris    1952-11-10                Hutchinson Inc   
...          ...       ...           ...                           ...   
15313    Stephen    Barker    1981-05-01             Marshall-Campbell   
15314    Stephen       Fox    1961-04-24             Williams-Phillips   
15315    Stephen   Jenkins    1943-08-09  Thornton, Carroll and Newman   
15316    Stephen     Smith    1948-08-20               Robinson-Harris   
15319    Stephen    Duncan    1994-03-27    Phillips, Brown and Powell   

                      email_address  \
1      rhodesclifford@henderson.com   
2      glen98@bryant-marshall.co.

In [96]:
uk.loc[:,'phone_number'] = uk['phone_number'].str.replace(r'(\+\d{2})|\s+|[(|)]','', regex= True)
uk.loc[:,'valid_phone'] = uk['phone_number'].str.len() == 11
print(uk['valid_phone'].value_counts())

valid_phone
True     6474
False    2897
Name: count, dtype: int64


C:\Users\Janoh\AppData\Local\Temp\ipykernel_2852\1538497412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uk.loc[:,'valid_phone'] = uk['phone_number'].str.len() == 11


In [97]:
uk.loc[~uk['phone_number'].apply(lambda x: len(str(x)) == 11), 'phone_number'] = np.nan
print(uk)

      first_name last_name date_of_birth                       company  \
1            Guy     Allen    1940-12-01                       Fox Ltd   
2          Harry  Lawrence    1995-08-02     Johnson, Jones and Harris   
3         Darren   Hussain    1972-09-23                   Wheeler LLC   
4          Garry     Stone    1952-12-20                    Warner Inc   
6           Anne    Morris    1952-11-10                Hutchinson Inc   
...          ...       ...           ...                           ...   
15313    Stephen    Barker    1981-05-01             Marshall-Campbell   
15314    Stephen       Fox    1961-04-24             Williams-Phillips   
15315    Stephen   Jenkins    1943-08-09  Thornton, Carroll and Newman   
15316    Stephen     Smith    1948-08-20               Robinson-Harris   
15319    Stephen    Duncan    1994-03-27    Phillips, Brown and Powell   

                      email_address  \
1      rhodesclifford@henderson.com   
2      glen98@bryant-marshall.co.

### Data cleaning of card details from the S3 bucket

In [99]:
import tabula

In [100]:
pdf = tabula.read_pdf('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf', pages = 'all')

In [101]:
len(pdf)

279

In [102]:
import pandas as pd
pdf2 = pd.concat(pdf, ignore_index = True)
print(pdf2.head(5))
print(type(pdf2))

        card_number expiry_date                card_provider  \
0    30060773296197       09/26  Diners Club / Carte Blanche   
1   349624180933183       10/23             American Express   
2  3529023891650490       06/23                 JCB 16 digit   
3   213142929492281       09/27                 JCB 15 digit   
4      502067329974       10/25                      Maestro   

  date_payment_confirmed  
0             2015-11-25  
1             2001-06-18  
2             2000-12-26  
3             2011-02-12  
4             1997-03-13  
<class 'pandas.core.frame.DataFrame'>


In [103]:
# Now in dataframe format
print(pdf2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15309 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15309 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15309 non-null  object
dtypes: object(4)
memory usage: 478.5+ KB
None


In [104]:
card_names = pdf2['card_provider'].value_counts()
get_card_names = card_names[card_names > 1000].index.tolist()
print(get_card_names)

['VISA 16 digit', 'JCB 16 digit', 'VISA 13 digit', 'JCB 15 digit', 'VISA 19 digit', 'Diners Club / Carte Blanche', 'American Express', 'Maestro', 'Discover', 'Mastercard']


In [105]:
#All the dodgy cards have dodgy data, so remove
checking_dodgy_cards = pdf2[~pdf2['card_provider'].isin(get_card_names)]
print(checking_dodgy_cards)
print(checking_dodgy_cards.info())

      card_number expiry_date card_provider date_payment_confirmed
377          NULL        NULL          NULL                   NULL
827    VAB9DSB8ZM  NWS3P2W38H    NB71VBAHJE             GTC9KBWJO9
847          NULL        NULL          NULL                   NULL
884          NULL        NULL          NULL                   NULL
1443   MOZOT5Q95V  8YJ3TYH6Z5    WJVMUO4QX6             DJIXF1AFAZ
2418         NULL        NULL          NULL                   NULL
2489         NULL        NULL          NULL                   NULL
2830         NULL        NULL          NULL                   NULL
3694   K0084A9R99  ACT9K6ECRJ    JRPRLPIBZ2             H2PCQP4W50
4196         NULL        NULL          NULL                   NULL
4208   Y8ITI33X30  WDWMN9TU45    TS8A81WFXV             XTD27ANR5Q
4916   RNSCD8OCIM  VNLNMWPJII    JCQMU8FN85             7VGB4DA1WI
5686         NULL        NULL          NULL                   NULL
6024         NULL        NULL          NULL                   

In [106]:
# pdf3 symbolises the removal of nulls and dodgy data
pdf3 = pdf2[pdf2['card_provider'].isin(get_card_names)]
print(pdf3['card_provider'].value_counts())

card_provider
VISA 16 digit                  2426
JCB 16 digit                   2403
VISA 13 digit                  1382
JCB 15 digit                   1363
VISA 19 digit                  1349
Diners Club / Carte Blanche    1312
American Express               1297
Maestro                        1281
Discover                       1260
Mastercard                     1211
Name: count, dtype: int64


In [107]:
print(pdf3['expiry_date'])

0        09/26
1        10/23
2        06/23
3        09/27
4        10/25
         ...  
15304    12/28
15305    11/24
15306    04/24
15307    06/27
15308    02/30
Name: expiry_date, Length: 15284, dtype: object


In [108]:
#pdf3['expiry_date'] = pdf3['expiry_date'].apply(parse)
#pdf3['expiry_date'] = pd.to_datetime(pdf3['expiry_date'], errors='coerce', format = '%m/%Y')
#na_index = pdf3.index[pdf3['date_of_birth'].isna()].tolist() 
pdf3['expiry_date'].head(20)

0     09/26
1     10/23
2     06/23
3     09/27
4     10/25
5     11/23
6     07/27
7     02/29
8     02/24
9     07/23
10    10/28
11    11/27
12    11/31
13    10/29
14    01/29
15    01/26
16    02/32
17    10/26
18    08/28
19    09/32
Name: expiry_date, dtype: object

In [109]:
def custom_parse(date_string):
    month,year = date_string.split('/')
    return parse('01'+'/'+ date_string, ignoretz = True)

In [110]:
pdf3['expiry_date'][1]

'10/23'

In [111]:
pdf3.loc[:,'expiry_date'] = pdf3['expiry_date'].apply(custom_parse)
pdf3.loc[:,'expiry_date'] = pd.to_datetime(pdf3['expiry_date'])

In [112]:
print(pdf3['expiry_date'].head(20))
print(pdf3['expiry_date'].isnull().sum())


0     2026-01-09 00:00:00
1     2023-01-10 00:00:00
2     2023-01-06 00:00:00
3     2027-01-09 00:00:00
4     2025-01-10 00:00:00
5     2023-01-11 00:00:00
6     2027-01-07 00:00:00
7     2029-01-02 00:00:00
8     2024-01-02 00:00:00
9     2023-01-07 00:00:00
10    2028-01-10 00:00:00
11    2027-01-11 00:00:00
12    2031-01-11 00:00:00
13    2029-01-10 00:00:00
14    2029-01-01 00:00:00
15    2026-01-01 00:00:00
16    2032-01-02 00:00:00
17    2026-01-10 00:00:00
18    2028-01-08 00:00:00
19    2032-01-09 00:00:00
Name: expiry_date, dtype: object
0


In [113]:
#No expiry dates our of the ordinary
expired = pdf3[pdf3['expiry_date'] > pd.to_datetime('2050-01-01')]
print(expired)

Empty DataFrame
Columns: [card_number, expiry_date, card_provider, date_payment_confirmed]
Index: []


In [114]:
pdf3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15284 non-null  object
 1   expiry_date             15284 non-null  object
 2   card_provider           15284 non-null  object
 3   date_payment_confirmed  15284 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


In [115]:
# date of payment looks good too
pdf3['date_payment_confirmed'].head(5)
pdf3.loc[:,'date_payment_confirmed'] = pdf3['date_payment_confirmed'].apply(parse)
pdf3['date_payment_confirmed'].isnull().sum()
pdf3[pdf3['date_payment_confirmed'] > pd.to_datetime("2022-12-01")]

,card_number,expiry_date,card_provider,date_payment_confirmed


In [116]:
# Card number
pdf3['card_number'].head(20)
print(type(pdf3['card_number'][1]))

<class 'int'>


In [117]:
def check_alpha(string):
    return str(string).isalpha()

In [118]:
#This didnt pick up some card numbers had '?' 
pdf3['card_number'].apply(check_alpha).value_counts()

card_number
False    15284
Name: count, dtype: int64

In [119]:
pdf3.loc[:,'card_number'] = str(pdf3['card_number'])
print(pdf3['card_provider'].value_counts())
# diners club = 14
# AMEX = 15
# Maestro = 12 to 19. 50, 56-69, 67
# Discover = 16
# Mastercard = 16

card_provider
VISA 16 digit                  2426
JCB 16 digit                   2403
VISA 13 digit                  1382
JCB 15 digit                   1363
VISA 19 digit                  1349
Diners Club / Carte Blanche    1312
American Express               1297
Maestro                        1281
Discover                       1260
Mastercard                     1211
Name: count, dtype: int64


In [120]:
def get_length(column):
    if 'Diners' in column:
        return 14
    elif 'American Express' in column:
        return 15
    elif 'Discover' in column:
        return 16
    elif 'Mastercard' in column:
        return 16
    elif 'JCB' in column or 'VISA' in column:
        return int(''.join(filter(str.isdigit, column)))
    elif 'Maestro' in column:
        return list(range(12,20))
    elif 'Diners Club / Carte Blanche' in column:
        return 14
    else:
        return None

In [121]:
pdf3.loc[:,'expected_num_dig'] = pdf3['card_provider'].apply(get_length)

C:\Users\Janoh\AppData\Local\Temp\ipykernel_2852\3089059518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf3.loc[:,'expected_num_dig'] = pdf3['card_provider'].apply(get_length)


In [122]:
#Run this code if you encounter any errors 
#pdf3 = pdf2[pdf2['card_provider'].isin(get_card_names)]

In [123]:
pdf3[['card_number','card_provider','expected_num_dig']]

,card_number,card_provider,expected_num_dig
0,0 30060773296197\n1 349624180...,Diners Club / Carte Blanche,14
1,0 30060773296197\n1 349624180...,American Express,15
2,0 30060773296197\n1 349624180...,JCB 16 digit,16
3,0 30060773296197\n1 349624180...,JCB 15 digit,15
4,0 30060773296197\n1 349624180...,Maestro,"[12, 13, 14, 15, 16, 17, 18, 19]"
...,...,...,...
15304,0 30060773296197\n1 349624180...,JCB 15 digit,15
15305,0 30060773296197\n1 349624180...,JCB 15 digit,15
15306,0 30060773296197\n1 349624180...,JCB 16 digit,16
15307,0 30060773296197\n1 349624180...,VISA 16 digit,16


In [124]:
pdf3.loc[:,'comparison'] = pdf3.apply(lambda row: len(str(row['card_number'])) == row['expected_num_dig'] if isinstance(row['expected_num_dig'], int) else len(str(row['card_number'])) in row['expected_num_dig'], axis=1)

C:\Users\Janoh\AppData\Local\Temp\ipykernel_2852\284436329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf3.loc[:,'comparison'] = pdf3.apply(lambda row: len(str(row['card_number'])) == row['expected_num_dig'] if isinstance(row['expected_num_dig'], int) else len(str(row['card_number'])) in row['expected_num_dig'], axis=1)


In [125]:
pdf3['comparison'].value_counts()

comparison
False    15284
Name: count, dtype: int64

In [126]:
import pandas as pd
review = pdf3[pdf3['comparison'] == False].head(100)
review.to_csv('review.csv')

In [127]:
review = pdf3[pdf3['card_provider'] == 'Maestro']
print(review)

                                             card_number          expiry_date  \
4      0          30060773296197\n1         349624180...  2025-01-10 00:00:00   
9      0          30060773296197\n1         349624180...  2023-01-07 00:00:00   
35     0          30060773296197\n1         349624180...  2027-01-08 00:00:00   
42     0          30060773296197\n1         349624180...  2027-01-07 00:00:00   
49     0          30060773296197\n1         349624180...  2025-01-02 00:00:00   
...                                                  ...                  ...   
15240  0          30060773296197\n1         349624180...  2032-01-04 00:00:00   
15285  0          30060773296197\n1         349624180...  2024-01-10 00:00:00   
15289  0          30060773296197\n1         349624180...  2026-01-11 00:00:00   
15290  0          30060773296197\n1         349624180...  2028-01-11 00:00:00   
15291  0          30060773296197\n1         349624180...  2028-01-05 00:00:00   

      card_provider date_pa

In [128]:
pdf3['card_number'] = pdf3['card_number'].astype(str)

C:\Users\Janoh\AppData\Local\Temp\ipykernel_2852\3786287763.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf3['card_number'] = pdf3['card_number'].astype(str)


In [129]:
pdf3[pdf3['card_number'].str.contains(r'\D')]


,card_number,expiry_date,card_provider,date_payment_confirmed,expected_num_dig,comparison
0,0 30060773296197\n1 349624180...,2026-01-09 00:00:00,Diners Club / Carte Blanche,2015-11-25 00:00:00,14,False
1,0 30060773296197\n1 349624180...,2023-01-10 00:00:00,American Express,2001-06-18 00:00:00,15,False
2,0 30060773296197\n1 349624180...,2023-01-06 00:00:00,JCB 16 digit,2000-12-26 00:00:00,16,False
3,0 30060773296197\n1 349624180...,2027-01-09 00:00:00,JCB 15 digit,2011-02-12 00:00:00,15,False
4,0 30060773296197\n1 349624180...,2025-01-10 00:00:00,Maestro,1997-03-13 00:00:00,"[12, 13, 14, 15, 16, 17, 18, 19]",False
...,...,...,...,...,...,...
15304,0 30060773296197\n1 349624180...,2028-01-12 00:00:00,JCB 15 digit,1997-06-06 00:00:00,15,False
15305,0 30060773296197\n1 349624180...,2024-01-11 00:00:00,JCB 15 digit,2004-06-16 00:00:00,15,False
15306,0 30060773296197\n1 349624180...,2024-01-04 00:00:00,JCB 16 digit,2020-02-05 00:00:00,16,False
15307,0 30060773296197\n1 349624180...,2027-01-06 00:00:00,VISA 16 digit,2008-06-16 00:00:00,16,False


In [130]:
pdf3.loc[pdf3['card_number'].str.contains(r'\D'), 'card_number'] = None

# Checking code to GET API data

In [131]:
#
import yaml
import requests

with open('db_creds.yaml','r') as file:
    credentials = yaml.safe_load(file)

response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=credentials['header'])

if response.status_code == 200:
    data = response.json()
else:
    print(f"Request failed with status code: {response.status_code}")
    print(f"Response Text: {response.text}")


In [132]:
print(data)

{'statusCode': 200, 'number_stores': 451}


In [133]:
import pandas as pd

with open('db_creds.yaml', 'r') as file:
    credentials = yaml.safe_load(file)

data_list = []
for store_number in range(1, 452):
    response = requests.get(f"https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{store_number}", headers=credentials['header'])

    if response.status_code == 200:
        data = response.json()
        data_list.append(data)
        # Process the data as needed for each store
        print(f"Data for store {store_number}: {data}")
    else:
        print(f"Request for store {store_number} failed with status code: {response.status_code}")
        print(f"Response Text: {response.text}")

df_api = pd.DataFrame(data_list)

Data for store 1: {'index': 1, 'address': 'Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, High Wycombe', 'longitude': '51.62907', 'lat': None, 'locality': 'High Wycombe', 'store_code': 'HI-9B97EE4E', 'staff_numbers': '34', 'opening_date': '1996-10-25', 'store_type': 'Local', 'latitude': '-0.74934', 'country_code': 'GB', 'continent': 'Europe'}
Data for store 2: {'index': 2, 'address': 'Heckerstraße 4/5\n50491 Säckingen, Landshut', 'longitude': '48.52961', 'lat': None, 'locality': 'Landshut', 'store_code': 'LA-0772C7B9', 'staff_numbers': '92', 'opening_date': '2013-04-12', 'store_type': 'Super Store', 'latitude': '12.16179', 'country_code': 'DE', 'continent': 'Europe'}
Data for store 3: {'index': 3, 'address': '5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury', 'longitude': '51.26', 'lat': None, 'locality': 'Westbury', 'store_code': 'WE-1DE82CEE', 'staff_numbers': '69', 'opening_date': '2014-01-02', 'store_type': 'Super Store', 'latitude': '-2.1875', 'country_code': 'GB', 'continent': 'E

KeyboardInterrupt: 

# Reviewing and cleaning the API data

In [346]:
api_data = pd.read_csv("api_data.csv")

In [347]:
api_data.head(2)

,Unnamed: 0,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,NaN,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
1,1,2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,NaN,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe


In [348]:
api_data =api_data.drop(api_data.columns[0],axis=1)

In [349]:
# Is lat an irrelevant empty column? yes
api_data['lat'].value_counts()

lat
13KJZ890JH    1
2XE1OWOC23    1
OXVE5QR07O    1
VKA5I8H32X    1
LACCWDI0SB    1
A3O5CBWAMD    1
UXMWDMX1LC    1
Name: count, dtype: int64

In [350]:
api_data =api_data.drop('lat',axis = 1)

In [351]:
#Randomly generated rows exist, use a numbers column to dig these out
def check_alpha(string):
    return str(string).isalpha()

random = api_data[api_data['country_code'].str.len() > 2]

In [146]:
print(random)

     Unnamed: 0  index     address   longitude         lat    locality  \
62           62     63  6FWDZHD7PW  1ZVU03X2P6  13KJZ890JH  9IBH8Y4Z0S   
171         171    172  UBCIFQLSNY  Q1TJY8H1ZH  2XE1OWOC23  1T6B406CI8   
230         230    231  RC99UKMZB2  2YBZ1440V6  OXVE5QR07O  6LVWPU1G64   
332         332    333  X349GIDWKU  O7NF1FZ74Y  VKA5I8H32X  RX9TCP2RGB   
380         380    381  ZBGB54ID4H  SKO4NMRNNF  LACCWDI0SB  CQMHKI78BX   
413         413    414  XTUAV57DP4  ID819KG3X5  A3O5CBWAMD  RY6K0AUE7F   
446         446    447  K0ODETRLS3  K8CXLZDP07  UXMWDMX1LC  3VHFDNP8ET   

     store_code staff_numbers opening_date  store_type    latitude  \
62   NRQKZWJ9OZ    BIP8K8JJW2   ZCXWWKF45G  QP74AHEQT0  1CJ5OAU4BR   
171  QIUU9SVP51    SKBXAXF5G5   7AHXLXIUEF  O0QJIRC943  3ZR3F89D97   
230  Y8J0Z2W8O9    2429OB3LMM   0OLAK2I6NS  50IB01SFAZ  L13EQEQODP   
332  ISEE8A57FE    74BY7HSB6P   A3PMVM800J  0RSNUU3DF5  J3BPB68Z1J   
380  T0R2CQBDUS    GT1FO6YGD4   GMMB02LA9V  B4KVQB3P5Y  A

In [352]:
# These rows have been found and will be dropped
api_data = api_data[api_data['country_code'].str.len() <= 2]

In [164]:
api_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 440 entries, 0 to 449
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          440 non-null    int64 
 1   address        440 non-null    object
 2   longitude      440 non-null    object
 3   locality       440 non-null    object
 4   store_code     440 non-null    object
 5   staff_numbers  440 non-null    object
 6   opening_date   440 non-null    object
 7   store_type     440 non-null    object
 8   latitude       440 non-null    object
 9   country_code   440 non-null    object
 10  continent      440 non-null    object
dtypes: int64(1), object(10)
memory usage: 41.2+ KB


In [233]:
#no nulls
api_data.isnull().sum()

index            0
address          0
longitude        0
locality         0
store_code       0
staff_numbers    0
opening_date     0
store_type       0
latitude         0
country_code     0
continent        0
dtype: int64

In [291]:
#Remove new line from address and replace with commas
api_data['address'] = api_data['address'].str.replace('\n', ',')

In [292]:
api_data['locality'].value_counts()

locality
Chapletown       14
Belper           13
Bushey           12
Exeter           11
Arbroath         10
                 ..
Sicklerville      1
Brierley Hill     1
Searcy            1
Port Richmond     1
Westchester       1
Name: count, Length: 116, dtype: int64

In [293]:
#Store code looks good
print(api_data['store_code'])
pattern = r'^[A-Z]{2}-[0-9A-F]{8}$'
unmatched_strings = api_data[~api_data['store_code'].str.contains(pattern)]


0      HI-9B97EE4E
1      LA-0772C7B9
2      WE-1DE82CEE
3      BE-18074576
4      GA-CAD01AC2
          ...     
444    CO-473A9FBB
445    KI-78096E8C
447    HI-EEA7AE62
448    BR-662EC74C
449    BA-B4AED588
Name: store_code, Length: 440, dtype: object


In [294]:
print(unmatched_strings)

Empty DataFrame
Columns: [index, address, longitude, locality, store_code, staff_numbers, opening_date, store_type, latitude, country_code, continent]
Index: []


In [295]:
# Opening date
date_check = api_data.copy()

In [296]:
date_check['opening_date']

0      1996-10-25
1      2013-04-12
2      2014-01-02
3      2019-09-09
4      1995-05-15
          ...    
444    2008-06-08
445    2005-05-12
447    1998-05-14
448    2020-10-17
449    2001-05-12
Name: opening_date, Length: 440, dtype: object

In [306]:
date_check['opening_date'] = pd.to_datetime(date_check['opening_date'], format = 'mixed')

In [308]:
null_index = date_check.loc[date_check['opening_date'].isnull()].index

In [309]:
api_data.loc[null_index]['opening_date']

Series([], Name: opening_date, dtype: object)

In [307]:
date_check.loc[null_index]['opening_date']

9     2012-10-08
10    2015-07-14
93    2022-01-20
121   2020-02-01
142   2003-05-27
189   2016-11-25
205   2008-12-07
241   2006-10-04
291   2001-05-04
339   1994-11-24
368   2009-02-28
393   2015-03-02
Name: opening_date, dtype: datetime64[ns]

In [310]:
api_data['opening_date'] = pd.to_datetime(api_data['opening_date'], format = 'mixed')

In [311]:
api_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 440 entries, 0 to 449
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          440 non-null    int64         
 1   address        440 non-null    object        
 2   longitude      440 non-null    object        
 3   locality       440 non-null    object        
 4   store_code     440 non-null    object        
 5   staff_numbers  440 non-null    object        
 6   opening_date   440 non-null    datetime64[ns]
 7   store_type     440 non-null    object        
 8   latitude       440 non-null    object        
 9   country_code   440 non-null    object        
 10  continent      440 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 57.4+ KB


In [312]:
staff_number = api_data.copy()

In [325]:
#Throws error letters present
staff_number['staff_numbers'].astype(int)

0      34
1      92
2      69
3      35
4      36
       ..
444    94
445    61
447    33
448    35
449    36
Name: staff_numbers, Length: 440, dtype: int32

In [322]:
non_digit_records = staff_number[staff_number['staff_numbers'].str.contains(r'\D', regex=True)]
print(non_digit_records)

     index                                            address longitude  \
30      31  Flat 69,Suzanne walk,East Michelle,E80 8HS, Ea...  52.68333   
178    179          Girschnerweg 163,93597 Angermünde, Dahlem   52.4581   
247    248  7430 Howe Extensions Suite 299,Kellyside, WA 9...  39.71734   
340    341  Studio 8,Lydia groves,North Hilarymouth,IV4 6E...  50.79205   
374    375  Salzstraße 1/9,74209 Bad Freienwalde, Charlott...  52.53048   

                locality   store_code staff_numbers opening_date   store_type  \
30          East Dereham  EA-24B31935           J78   2012-11-09       Outlet   
178               Dahlem  DA-ACC520AE           30e   1994-03-07        Local   
247         Sicklerville  SI-C489938D           80R   1994-02-28       Outlet   
340             Southsea  SO-B5B9CB3B           A97   2018-05-08  Super Store   
374  Charlottenburg-Nord  CH-99475026           3n9   1995-03-05        Local   

      latitude country_code continent  
30     0.93333        

In [319]:
# didnt pick up that staff numbers had lettters
staff_number[staff_number['staff_numbers'].astype(str).str.len() > 3]

,index,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent


In [324]:
# Remove letters but keep the numbers in 
api_data['staff_numbers'] = api_data['staff_numbers'].replace(r'\D', '', regex=True)

In [326]:
api_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 440 entries, 0 to 449
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          440 non-null    int64         
 1   address        440 non-null    object        
 2   longitude      440 non-null    object        
 3   locality       440 non-null    object        
 4   store_code     440 non-null    object        
 5   staff_numbers  440 non-null    object        
 6   opening_date   440 non-null    datetime64[ns]
 7   store_type     440 non-null    object        
 8   latitude       440 non-null    object        
 9   country_code   440 non-null    object        
 10  continent      440 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 57.4+ KB


In [327]:
#Store type looks good
api_data['store_type'].value_counts()

store_type
Local          255
Super Store     89
Mall Kiosk      51
Outlet          45
Name: count, dtype: int64

In [328]:
255 + 89 +51 +45

440

In [334]:
api_data['latitude'].astype(float)


count          440
unique         116
top       -1.47217
freq            14
Name: latitude, dtype: object

In [353]:
api_data['longitude'].astype(float)

0      51.62907
1      48.52961
2      51.26000
3      53.02330
4      53.38333
         ...   
444    50.76306
445    52.20000
447    51.62907
448    49.03685
449    50.64336
Name: longitude, Length: 440, dtype: float64

In [335]:
api_data['country_code'].value_counts()

country_code
GB    265
DE    141
US     34
Name: count, dtype: int64

In [336]:
api_data['country_code'].value_counts().sum()

440

In [340]:
api_data['continent'].value_counts()

continent
Europe     406
America     34
Name: count, dtype: int64

In [339]:
api_data['continent'] = api_data['continent'].str.replace('ee', '')

### Extracting S3 object using boto3

In [2]:
# Downloading S3 object using boto3
# import boto3
# s3 = boto3.client('s3')
# s3.download_file('data-handling-public','products.csv','D:/vs_ipynb/learning-material/multinational-retail/products.csv')

In [3]:
# import pandas as pd
# df_csv = pd.read_csv('products.csv',index_col = 0)

In [11]:
print(df_csv)

                                     product_name product_price  weight  \
0     FurReal Dazzlin' Dimples My Playful Dolphin        £39.99   1.6kg   
1             Tiffany's World Day Out At The Park        £12.99  0.48kg   
2             Tiffany's World Pups Picnic Playset         £7.00    590g   
3        Tiffany's World Wildlife Park Adventures        £12.99    540g   
4                         Cosatto Cosy Dolls Pram        £30.00  1.91kg   
...                                           ...           ...     ...   
1848    Goodmans Qi Autosense In Car Phone Holder        £15.00    125g   
1849    Goodmans Qi Wireless Charging Phone Stand        £15.00    134g   
1850                     RAC 12V Wet & Dry Vacuum        £18.00    820g   
1851    Goodmans Qi Autosense In Car Phone Holder        £15.00    125g   
1852               Beldray Telescopic Ladder 2.9m        £69.00   9.4kg   

            category            EAN  date_added  \
0     toys-and-games  7425710935115  2005-12-02 

In [57]:
import boto3
from io import BytesIO
import pandas as pd

# Initialize Boto 3 S3 client
s3 = boto3.client('s3')

# Define the bucket and object key
bucket_name = 'data-handling-public'
object_key = 'products.csv'

# Read the file from S3
try:
    response = s3.get_object(Bucket=bucket_name, Key=object_key)
    # Access and read the content of the file. Data type = Bytes due to get request
    data = response['Body'].read()

    # Process the data as needed
    df = pd.read_csv(BytesIO(data), index_col = 0)  # Replace this with your processing logic

except Exception as e:
    print(f"Error: {e}")

In [111]:
# Example S3 path
s3_path = "s3://data-handling-public/products.csv"

# Separating bucket and object
if s3_path.startswith("s3://"):
    path_without_protocol = s3_path[len("s3://"):]
    bucket, obj = path_without_protocol.rsplit('/', 1)

    print("Bucket:", bucket)
    print("Object:", obj)
else:
    print("Invalid S3 path.")

Bucket: data-handling-public
Object: products.csv


In [59]:
print(df.head(15))

                                   product_name product_price   weight  \
0   FurReal Dazzlin' Dimples My Playful Dolphin        £39.99    1.6kg   
1           Tiffany's World Day Out At The Park        £12.99   0.48kg   
2           Tiffany's World Pups Picnic Playset         £7.00     590g   
3      Tiffany's World Wildlife Park Adventures        £12.99     540g   
4                       Cosatto Cosy Dolls Pram        £30.00   1.91kg   
5                  Cocomelon Magnetic Scribbler        £12.99   0.91kg   
6                       Peppa Pig House Felties         £7.00   0.46kg   
7                     Paw Patrol Walkie Talkies        £12.99   0.38kg   
8                   Peppa Pig 3D Walkie Talkies        £12.99   0.38kg   
9          Disney Encanto Magical Casa Madrigal        £89.99  8.981kg   
10                Disney Encanto Mirabel's Room        £24.99  1.478kg   
11         Disney Encanto Isabela's Garden Room        £24.99     1.2g   
12          Disney Encanto Antonio's T

In [30]:
# Check for null
df.info() # 4

<class 'pandas.core.frame.DataFrame'>
Index: 1853 entries, 0 to 1852
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_name   1849 non-null   object
 1   product_price  1849 non-null   object
 2   weight         1849 non-null   object
 3   category       1849 non-null   object
 4   EAN            1849 non-null   object
 5   date_added     1849 non-null   object
 6   uuid           1849 non-null   object
 7   removed        1849 non-null   object
 8   product_code   1849 non-null   object
dtypes: object(9)
memory usage: 144.8+ KB


In [54]:
# null_values = df[df.isnull().any(axis=1)].index
# df.drop(null_values)
df.dropna(inplace=True)

In [53]:
# Likely to be random generated data
df['category'].value_counts()

category
homeware              1138
toys-and-games         257
food-and-drink         250
pets                    93
sports-and-leisure      40
health-and-beauty       40
diy                     28
Name: count, dtype: int64

In [49]:
cat = df['category'].value_counts()
get_cat = cat[cat < 2].index.tolist()
df[df['category'].isin(get_cat)]

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code


In [48]:
df = df[~df['category'].isin(get_cat)]
print(df['category'].value_counts())

category
homeware              1138
toys-and-games         257
food-and-drink         250
pets                    93
sports-and-leisure      40
health-and-beauty       40
diy                     28
Name: count, dtype: int64


In [36]:
# Find rows and columns containing '?'. Doesnt exist for once
question_mark_indices = df.map(lambda x: '?' in str(x)).any()
question_mark_rows = df[df.map(lambda x: '?' in str(x)).any(axis=1)].index

print("Columns with '?':")
print(question_mark_indices[question_mark_indices].index.tolist())
print("Rows with '?':")
print(question_mark_rows.tolist())

Columns with '?':
[]
Rows with '?':
[]


In [44]:
# null_values = df[df.isnull().any(axis=1)].index
# print(null_values)

Index([], dtype='int64')


In [55]:
df[['weight','units']] = df['weight'].str.extract('(\d*\.?\d+)(\D+)', expand=True)

In [51]:
df['units'].value_counts()

units
kg     954
g      853
 x      28
ml       9
g .      1
oz       1
Name: count, dtype: int64

In [70]:
df.loc[df.index[df['units'].str.contains('x', case=True, na=False)].tolist()]

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code,units
298,Pedigree Vital Protection Dog Food in Jelly 12...,£4.49,12,pets,2439834307647,1995-06-25,5ec5a431-7330-4d9e-bf3c-702fc85f6efe,Still_avaliable,d4-9698287C,x
299,Cesar Country Kitchen Dog Food 8 x 150g,£5.49,8,pets,5158002633117,1994-03-22,c379d810-61bf-4ed2-994c-0d3308d5be7b,Still_avaliable,M4-1688574Q,x
300,Pedigree Puppy Dog Food in Jelly 12pk,£4.49,12,pets,5242405808651,2014-05-20,40963728-bdae-48f1-beec-7c6dd0225921,Still_avaliable,J4-5517838n,x
301,Pedigree Dog Food in Gravy 12pk,£4.49,12,pets,6160542808008,2020-12-20,3109d9e8-56b5-4233-a682-0bcdb38182b0,Still_avaliable,v6-1209149g,x
302,Pedigree Dog Food in Jelly 12pk,£4.49,12,pets,3928398867180,2006-03-27,28625483-3723-4eaa-9f14-a1a16affbed8,Still_avaliable,C5-0179057G,x
303,Chappie Dog Food 6 x 412g,£5.99,6,pets,8155594260063,1993-06-30,6b86f83c-fccf-42eb-b1d5-3971915c18a5,Still_avaliable,Z4-535642u,x
304,Butcher's Puppy Meaty Chunks in Jelly Tins 6 x...,£4.35,6,pets,9386802556862,2019-06-16,a21e8202-de56-40bf-aee9-1b65989b1030,Still_avaliable,x0-3876046p,x
305,Pedigree Vital Protection Dog Food in Jelly 12...,£4.49,12,pets,6045981116269,1996-10-12,39ce65ab-d873-4953-a363-af9a705ca1ef,Still_avaliable,O3-6314460b,x
381,Sheba Classics Ocean Selection in Terrine 8 x 85g,£3.49,8,pets,5864355297349,2014-01-01,96f4d10f-1cb0-4898-b170-8dcd54c78f05,Still_avaliable,I3-7208040s,x
382,Classic Jumbo Pack Cat Food in Jelly 40 x 100g,£10.99,40,pets,3610175997493,2003-09-03,003e2c32-2d88-4ace-9d3c-97863fb62959,Still_avaliable,Y9-9993954b,x


In [39]:
def transform_weight(row):
    weight, unit = row['weight'],row['units']
    if unit == 'kg':
        return weight
    elif unit == 'g'or unit == 'g .':
        return weight / 1000 
    elif unit == 'ml':
        return weight / 1000
    elif unit == 'oz':
        return weight * 0.0283495
    else:
        return 0

In [56]:
df['weight'] = df.apply(transform_weight,axis=1)

In [43]:
df['weight'] = (df['weight']).astype(float)

C:\Users\Janoh\AppData\Local\Temp\ipykernel_18948\3355173983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weight'] = (df['weight']).astype(float)


In [44]:
print(df)

                                     product_name product_price  weight  \
0     FurReal Dazzlin' Dimples My Playful Dolphin        £39.99     0.0   
1             Tiffany's World Day Out At The Park        £12.99     0.0   
2             Tiffany's World Pups Picnic Playset         £7.00     0.0   
3        Tiffany's World Wildlife Park Adventures        £12.99     0.0   
4                         Cosatto Cosy Dolls Pram        £30.00     0.0   
...                                           ...           ...     ...   
1848    Goodmans Qi Autosense In Car Phone Holder        £15.00     0.0   
1849    Goodmans Qi Wireless Charging Phone Stand        £15.00     0.0   
1850                     RAC 12V Wet & Dry Vacuum        £18.00     0.0   
1851    Goodmans Qi Autosense In Car Phone Holder        £15.00     0.0   
1852               Beldray Telescopic Ladder 2.9m        £69.00     0.0   

            category            EAN  date_added  \
0     toys-and-games  7425710935115  2005-12-02 

In [86]:
df.drop('units', axis=1, inplace = True)

In [96]:
# More NAs
null_rows = df[df.isnull().any(axis=1)].index
print(null_rows)

Index([751, 1133, 1400], dtype='int64')


In [100]:
df.iloc[[750, 1130, 1397]]

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
751,VLPCU81M30,XCD69KUI0K,0,NaN,OO7KH8P79I,CCAVRB79VV,7QB0Z9EW1G,T3QRRH7SRP,SDAV678FVD
1133,9SX4G65YUX,N9D2BZQX63,0,NaN,E8EOGWOY8S,09KREHTMWL,CP8XYQVGGU,BPSADIOQOK,BSDTR67VD90
1400,LB3D71C025,ODPMASE7V7,0,NaN,BHPF2JTNKQ,PEPWA0NCVH,VIBLHHVPMN,H5N71TV8AY,OPSD21HN67


In [101]:
df.drop(null_rows, axis = 0, inplace = True)

In [102]:
df['date_added'] = pd.to_datetime(df['date_added'], format = 'mixed')

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1846 entries, 0 to 1852
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   product_name   1846 non-null   object        
 1   product_price  1846 non-null   object        
 2   weight         1846 non-null   float64       
 3   category       1846 non-null   object        
 4   EAN            1846 non-null   object        
 5   date_added     1846 non-null   datetime64[ns]
 6   uuid           1846 non-null   object        
 7   removed        1846 non-null   object        
 8   product_code   1846 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 144.2+ KB


In [106]:
df['removed'].value_counts()

removed
Still_avaliable    1752
Removed              94
Name: count, dtype: int64

In [109]:
df[~df['product_price'].str.contains('£', na=False)]

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
